<a href="https://colab.research.google.com/github/usseif97/MRNET-for-Knee-Diagnosis/blob/master/datagen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
def load_scan(root_path, task, view, index):
    index_extended = '0' * (4 - len(str(index))) + str(index)
    path = root_path + f"/{task}/{view}/{index_extended}.npy"
    print(f'loading example: {index}')
    x = np.load(path)
    x = np.stack((x,)*3, axis=3)
    return x

def load_view(root_path, task, view, start, end):
  data_size = end - start
  X = {}
  for i, index in enumerate(range(start, end)):
    X[i] = load_scan(root_path, task, view, index)
  return X

In [0]:
class MRNetDataGenerator(keras.utils.Sequence):
  def __init__(self, root_path = None, task = 'train', view = 'axial', result_label = 'abnormal', batch_size=1, shape=(256,256), n_channels=3,
                 shuffle=False, extractor = None, bounds = None, X = None):
    'Initialization'
    self.root_path = '/content/drive/My Drive/DataSet/MRNET data set/MRNet-v1.0'
    self.shape = shape
    self.batch_size = batch_size
    self.n_channels = n_channels
    self.shuffle = shuffle
    self.extractor = extractor
    self.task = task
    self.view = view
    self.bounds = bounds
    self.X = X
    if self.task != 'train':
      self.shuffle = False
    self.result_label = result_label
    records = pd.read_csv(self.root_path + f"/{self.task}-{self.result_label}.csv", header=None, names=['id', 'label'])
    self.Y = records['label'].to_numpy()
    if (self.task == 'train'):
      if self.bounds == None:
        self.num_examples = 1130
      else:
        self.num_examples = bounds[1] - bounds[0]
    else:
      self.num_examples = 120
    
    self.on_epoch_end()

  def on_epoch_end(self):
    'Updates indexes after each epoch'
    if self.task == 'train':
      if self.bounds != None:
        self.indexes = np.arange(self.bounds[0], self.bounds[1])
      else:
        self.indexes = np.arange(0, 1130)
    else:
      self.indexes = np.arange(1130, 1250)
  
    if self.shuffle == True:
      np.random.shuffle(self.indexes)

  def __load_scan(self, index):
    index_extended = '0' * (4 - len(str(index))) + str(index)
    path = self.root_path + f"/{self.task}/{self.view}/{index_extended}.npy"
    print(f'loading example: {index}')
    x = np.load(path)
    x = np.stack((x,)*self.n_channels, axis=1)
    return x

  def __load_features(self, index):
    index_extended = '0' * (4 - len(str(index))) + str(index)
    path = self.root_path + f"/{self.task}/{self.view}/{self.extractor}/{index_extended}.npy"
    x = np.load(path).reshape((1, 512))
    return x

  def __load_labels(self):
    records = pd.read_csv(self.root_path + f"/{self.task}-{self.result_label}.csv", header=None, names=['id', 'label'])
    self.Y = records['label'].to_numpy()
  
  def __getitem__(self, index):
    'Generate one batch of data'
    ind = self.indexes[index]
    if self.X == None:
      X = np.empty((self.batch_size, self.n_channels))
      if self.extractor == None:
        X = np.empty((self.batch_size, self.n_channels, *self.shape))
      else:
        X = np.empty((self.batch_size, self.shape))
        #X = tf.squeeze(X)
      #indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
      X = self.__load_scan(ind)
    else:
      X = self.X[ind]
      
    X = tf.expand_dims(X, 0)
    Y = self.Y[ind:ind+1]

  #  for i, ind in enumerate(indexes):
  #    #indexes[i] = '0' * (4 - len(str(num))) + str(num)
  #    if self.extractor == None:
  #      #X[i,] = self.__load_scan(ind)
  #      X = self.__load_scan(ind)
  #      #X = applications.vgg16.preprocess_input(X)
  #      X = tf.expand_dims(X, 0)
        
  #    else:
  #      X[i,] = self.__load_features(ind)

  #  Y = self.Y[index*self.batch_size:(index+1)*self.batch_size]

    return X, Y

  def __len__(self):
    'Denotes the number of batches per epoch'
    return int(np.floor(self.num_examples / self.batch_size))
 